---
title: "Airline Loyalty Causal Inference"
format: gfm
---


## Milestone 1: Project Idea

I love anything airline related, so my idea is to analyze some airline data to see if one thing causes another. Specifically, I want to see if having an airline's loyalty card increases the revenue that customer makes for the airline (called CLV or Customer Lifetime Value).

## Milestone 2: Data Story

Causal Inference Question:
Does having a higher loyalty card status (Star, Nova, Aurora) cause an increase in CLV?

Context:
I want to study if having a higher loyalty card status for an airline causes an increase in CLV. CLV is Customer Lifetime Value. It is computed as the sum of all revenues (or invoices) generated by the customer for their flight bookings over their entire membership period. Basically, it's how much total revenue a specific customer is expected to generate. The Star loyalty card is the highest, Nova is the next highest, and Aurora is third highest. In the data, only these 3 loyalties are listed.

The outcome would be positive and continuous values because it's how much revenue is expected to be made under the CLV. Other predictor variables that could influence CLV is salary, education level, marital status, total flights, and enrollment year/month. Other variables that could influence our outcome could be country of residence, gender, distance traveled, points redeemed, and dollar cost of points redeemed.

My story is that I believe that having a loyalty card increases the revenue you will make for the airline.

https://www.kaggle.com/datasets/agungpambudi/airline-loyalty-campaign-program-impact-on-flights/data


## Milestone 3: DAG

This is the first DAG I created.

![Initial Old DAG](../figures/Old_DAG.png)

This is the second DAG I created. I add more variables because I have most of these columns in my dataset.

![Second Iteration of Initial Old DAG](../figures/Old_DAG_Updated.png)

## Milestone 4: Identification Strategy

Based on the DAG, here are all possible paths (23 paths) from LCS (Loyalty Card Status) to CLV (Customer Lifetime Value):

* LCS, CLV
* LCS, I, CLV
* LCS, I, TF, CLV
* LCS, I, TF, FD, CLV
* LCS, I, TF, CMS, CLV
* LCS, I, TF, CMS, CE, CLV
* LCS, TF, CLV
* LCS, TF, I, CLV
* LCS, TF, FD, CLV
* LSC, TF, CMS, CLV
* LCS, TF, CMS, CE, CLV
* LCS, CMS, CLV
* LCS, CMS, CE, CLV
* LCS, CMS, TF, CLV
* LCS, CMS, TF, FD, CLV
* LCS, CMS, TF, I, CLV
* LCS, CE, CLV
* LCS, CE, CMS, CLV
* LCS, CE, CMS, TF, CLV
* LCS, CE, CMS, TF, FD, CLV
* LCS, CE, CMS, TF, I, CLV
* LCS, UP, CLV
* LCS, LTE, CLV

Direct pipes, Can estimate total causal effect through them

* LCS, UP, CLV
* LCS, LTE, CLV

Here are the backdoors & what to do about them:

* LCS, I, CLV  --  Fork, Condition on I
* LCS, I, TF, CLV -- Fork, Pipe, Condition on TF
* LCS, I, TF, FD, CLV -- Fork, Pipe, Pipe, Condition on FD (?)
* LCS, I, TF, CMS, CLV --Fork, Collider, Fork, Condition on CMS
* LCS, I, TF, CMS, CE, CLV -- Fork, Collider, Fork, Pipe, Condition on CE (??)
* LCS, TF, CLV -- Fork, Condition on TF
* LCS, TF, I, CLV -- Fork, Pipe, Condition on I
* LCS, TF, FD, CLV -- Fork, Pipe, Condition on FD (?)
* LSC, TF, CMS, CLV -- Pipe, Fork, Condition on CMS
* LCS, TF, CMS, CE, CLV -- Pipe, Fork, Pipe, Condition on CE
* LCS, CMS, CLV -- Fork, Condition on CMS
* LCS, CMS, CE, CLV -- Fork, Pipe, Condition on CE (?)
* LCS, CMS, TF, CLV -- Fork, Pipe, Condition on CMS (?)
* LCS, CMS, TF, FD, CLV -- Fork, Pipe, Pipe, Condition on CMS & TF (??)
* LCS, CMS, TF, I, CLV -- Fork, Collider, Fork, Condition on CMS, TF, & I (all are needed in the end I believe) (?) 
* LCS, CE, CLV -- Fork, Condition on CE
* LCS, CE, CMS, CLV -- Pipe, Fork, Condition on CE or CMS
* LCS, CE, CMS, TF, CLV -- Pipe, Fork, Pipe, Condition on CE or CMS or TF (all are needed in the end I believe) (?)
* LCS, CE, CMS, TF, FD, CLV -- Pipe, Fork, Pipe, Pipe, Condition on CE or CMS or TF (all are needed in the end I believe) (?)
* LCS, CE, CMS, TF, I, CLV -- Pipe, Fork, Collider, Fork, Condition on CE or CMS or TF or I (all are needed in the end I believe) (?)

Adjustment Set: I, TF, CMS, and CE.

## Milestone 5: Simulate Data and Recover Parameters


In [ ]:
import numpy as np
import polars as pl
import seaborn as sns
from sklearn.linear_model import LinearRegression

np.random.seed(42)

# Set the parameter values.
beta0 = 1000
income = 70000
flight_dist = 3500
travel_freq = 5
cust_marketing_strat = 1
cust_engagement = 3
loyalty_card_status = 1000
n = 100

sim_data = (
    # Simulate predictors using appropriate np.random distributions.
    pl.DataFrame({
        'x': np.random.uniform(0, 7, size=n),
        'flight_dist': np.random.uniform(0, 10000, size=n),
        'travel_freq': np.random.uniform(1, 10, size=n),
        'cust_marketing_strat': np.random.uniform(0, 5, size=n),
        'cust_engagement': np.random.uniform(1, 10, size=n),
        'loyalty_card_status': np.random.choice([0, 1], size=n)  # New binary predictor
    })
    # Use predictors and parameter values to simulate the outcome.
    .with_columns([ 
        (
            beta0 + income * pl.col('x') + flight_dist * pl.col('flight_dist') +
            travel_freq * pl.col('travel_freq') +
            cust_marketing_strat * pl.col('cust_marketing_strat') +
            cust_engagement * pl.col('cust_engagement') +
            loyalty_card_status * pl.col('loyalty_card_status') +  # Adding loyalty_card_status with a coefficient (e.g., 1000)
            np.random.normal(0, 3, size=n)
        ).alias('CLV')  # Renaming y to CLV
    ])
)

# Display the data
sim_data

# Visualize the data
# sns.scatterplot(data=sim_data, x='x', y='CLV')
# sns.lmplot(data=sim_data, x='x', y='CLV', height=6, aspect=1, scatter_kws={'s': 10}, line_kws={'color': 'red'})

# Specify the X matrix and CLV vector.
X = sim_data[['x', 'flight_dist', 'travel_freq', 'cust_marketing_strat', 'cust_engagement', 'loyalty_card_status']]
CLV = sim_data['CLV']

# Create a linear regression model.
model = LinearRegression(fit_intercept=True)
# Train the model.
model.fit(X, CLV)

# Print the coefficients
print(f'Intercept: {model.intercept_}')
print(f'Slope for x: {model.coef_[0]}')
print(f'Slope for flight_dist: {model.coef_[1]}')
print(f'Slope for travel_freq: {model.coef_[2]}')
print(f'Slope for cust_marketing_strat: {model.coef_[3]}')
print(f'Slope for cust_engagement: {model.coef_[4]}')
print(f'Slope for loyalty_card_status: {model.coef_[5]}')

# Have you recovered the parameters?
# Yes

In this code, a simulated dataset is created using various predictors (independent variables) to estimate the Customer Lifetime Value (CLV), which is the outcome (dependent variable). The code first defines the parameter values for each predictor, including factors like income, flight distance, travel frequency, customer marketing strategy, customer engagement, and loyalty card status. It then generates random values for these predictors and combines them in a linear equation to simulate CLV. This equation also includes a small amount of random noise to make the data more realistic.

After generating the data, a linear regression model is applied to it using the `LinearRegression` function from the `sklearn` library. The model is trained using the predictors (`X`) and the simulated CLV values. Once the model is trained, the code prints the estimated coefficients for each predictor, which represent how each variable impacts the CLV. By examining these coefficients, we can understand the relationship between the predictors and CLV.

## Milestone 6: Exploratory Data Analysis

Loyalty Card Status is whether or not you have an airline loyalty card, whether that's Star, Nova, or Aurora.
Here is a break down of the data by type of loyalty card. We can see that most people have the highest loyalty card, Star, followed by Nova and then by Aurora.


In [ ]:
#| eval: false
# Create a bar plot for loyalty_card_status
ax = sns.countplot(x='Loyalty Card', data=data)

# Set labels and title
plt.xlabel('Loyalty Card Status')
plt.ylabel('Count')
plt.title('Distribution of Loyalty Card Status')

# Add the count labels on top of the bars without decimals
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                fontsize=12, color='black', 
                xytext=(0, 5), textcoords='offset points')

# Save the figure
plt.savefig('../figures/loyalty_card_distribution_bar_chart.png', dpi=300, bbox_inches='tight')

![Loyalty Card Status Bar Chart](../figures/loyalty_card_distribution_bar_chart.png)

Most people have an income between roughly $80,000 and $125,000. This is expected. There are of course some who have higher salaries, hence the right skew.


In [ ]:
#| eval: false
# Clean the data by removing null, empty, and negative salary values
data_clean = data[data['Salary'].notnull() & (data['Salary'] != '')]
data_clean = data_clean[data_clean['Salary'] >= 0]

# Plot the histogram
plt.hist(data_clean['Salary'], bins=8, color='skyblue', edgecolor='black')
plt.title('Histogram of Salary')
plt.xlabel('Salary')
plt.ylabel('Frequency')

# Save the figure
plt.savefig('../figures/salary_histogram.png', dpi=300, bbox_inches='tight')

![Salary Histogram](../figures/salary_histogram.png)

Most people have an income between roughly $80,000 and $125,000. This is expected. There are of course some who have higher salaries, hence the right skew.


In [ ]:
#| eval: false
# Clean the data by removing null, empty, and negative salary values
data_clean = data[data['Salary'].notnull() & (data['Salary'] != '')]
data_clean = data_clean[data_clean['Salary'] >= 0]

# Plot the histogram
plt.hist(data_clean['Salary'], bins=8, color='skyblue', edgecolor='black')
plt.title('Histogram of Salary')
plt.xlabel('Salary')
plt.ylabel('Frequency')

# Save the figure
plt.savefig('../figures/salary_histogram.png', dpi=300, bbox_inches='tight')

![Salary Histogram](../figures/salary_histogram.png)

Travel Frequency is how frequently a person travels. I do not have this variable in my dataset.

Customer Marketing Strategy is how well the airline markets their products. I don't have a direct variable for this in my dataset but I do know which enrollment types people did when they got a loyalty card.


In [ ]:
#| eval: false
# Create a bar plot for Enrollment Type
ax = sns.countplot(x='Enrollment Type', data=data)

# Set labels and title
plt.xlabel('Enrollment Type')
plt.ylabel('Count')
plt.title('Distribution of Enrollment Type')

# Add the count labels on top of the bars without decimals
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', 
                fontsize=12, color='black', 
                xytext=(0, 5), textcoords='offset points')

# Save the figure
plt.savefig('../figures/enrollment_type_bar_chart.png', dpi=300, bbox_inches='tight')

![Salary Histogram](../figures/enrollment_type_bar_chart.png)

Customer Engagement is how engaged a person is in the airline. I do not have this variable in my dataset.

CLV is Customer Lifetime Value, or how much revenue a single person generates the airline company. It seems that most people earn the airline between roughly $2,000 and $15,000.


In [ ]:
#| eval: false
# Clean the data and remove negative values
data_clean = data[data['CLV'].notnull() & (data['CLV'] != '')]
data_clean = data_clean[data_clean['CLV'] >= 0]  

# Plot the histogram
plt.hist(data_clean['CLV'], bins=6, edgecolor='black')

# Add titles and labels
plt.title('Distribution of Customer Lifetime Value (CLV)')
plt.xlabel('Customer Lifetime Value (CLV)')
plt.ylabel('Count')

# Save the figure
plt.savefig('../figures/clv_histogram.png', dpi=300, bbox_inches='tight')

![Salary Histogram](../figures/clv_histogram.png)


## Milestone 7: Estimate Causal Effects


In [ ]:
# eval: false
import numpy as np
import polars as pl
import pymc as pm
import arviz as az
import seaborn as sns

np.random.seed(42)

# Set the parameter values.
beta0 = 1000
income = 70000
flight_dist = 3500
travel_freq = 5
cust_marketing_strat = 1
cust_engagement = 3
loyalty_card_status = 1000
n = 100

# Simulate Data
sim_data = (
    pl.DataFrame({
        'x': np.random.uniform(0, 7, size=n),
        'flight_dist': np.random.uniform(0, 10000, size=n),
        'travel_freq': np.random.uniform(1, 10, size=n),
        'cust_marketing_strat': np.random.uniform(0, 5, size=n),
        'cust_engagement': np.random.uniform(1, 10, size=n),
        'loyalty_card_status': np.random.choice([0, 1], size=n)
    })
    .with_columns([ 
        (
            beta0 + income * pl.col('x') + flight_dist * pl.col('flight_dist') +
            travel_freq * pl.col('travel_freq') +
            cust_marketing_strat * pl.col('cust_marketing_strat') +
            cust_engagement * pl.col('cust_engagement') +
            loyalty_card_status * pl.col('loyalty_card_status') +
            np.random.normal(0, 3, size=n)
        ).alias('CLV')
    ])
)

# Separate predictors and outcome
X = sim_data[['x', 'flight_dist', 'travel_freq', 'cust_marketing_strat', 'cust_engagement', 'loyalty_card_status']].to_numpy()
CLV = sim_data['CLV'].to_numpy()

# Bayesian Linear Regression Model
with pm.Model() as clv_model:
    # Priors
    alpha = pm.Normal('alpha', mu=0, sigma=1000)
    beta = pm.Normal('beta', mu=0, sigma=50000, shape=X.shape[1])  # One coefficient per feature
    sigma = pm.HalfNormal('sigma', sigma=100)

    # Likelihood
    mu = alpha + X @ beta  # Matrix multiplication of predictors and coefficients
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=CLV)

    # Posterior Sampling
    trace = pm.sample(1000, return_inferencedata=True)

# Summary of results
summary = az.summary(trace, round_to=2)
print(summary)

# Visualizing the marginal posteriors
az.plot_trace(trace, combined=True)


# Save the figure as a file
plt.savefig("trace_plot.png", dpi=300, bbox_inches="tight")

# Show the plot (optional)
plt.show()

![trace plot](../figures/trace_plot.png)


## Milestone 8: Intermediate Presentation

See my intermediate presentation [Intermediate Presentation Slides](https://github.com/marcdotson/causal-inference/blob/main/presentations/multivariate-models.html). To summarize some feedback:

- Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.
- Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.
- Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.
- Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.



## Milestone 9: Run Conjoint Experiment



![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Ever%20been%20on%20a%20plane_%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Employed%20rn_%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%2018_plus_%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20How%20often%20do%20you%20fly_%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Travel%20Reason_%20Chart.png)

![DAG](../figures/Loyalty%20Card%20Package%20Options%20-%20Card%20Type%20Chart.png)

![DAG](../figures/Loyalty%20Card%20Package%20Options%20-%20Benefit%201%20Chart.png)

![DAG](../figures/Loyalty%20Card%20Package%20Options%20-%20Benefit%202%20Chart.png)

![DAG](../figures/Loyalty%20Card%20Package%20Options%20-%20Price%20Chart.png)

![DAG](../figures/Loyalty%20Card%20Package%20Options%20-%20Attribute%20importance%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Buy%20a%20card_%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Yearly%20Income%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Education%20Level%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Age%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Gender%20Chart.png)

![DAG](../figures/Airline%20Loyalty%20Conjoint%20Survey%20-%20Married_%20Chart.png)



## Milestone 10: Implement Diff-in-Diff Strategy


## Milestone 11: Clean Up Project Report

I created and cleaned up my project report in the writing folder.

The CLV histogram now have bins that touch and a more appropriate number of bins.

The Salary histogram no longer has a bin below zero (since you can't have a negative salary).

I updated my DAG along the way.

![DAG](../figures/DAG_CLV.jpg)